In [1]:
%load_ext autoreload
%autoreload 2
import sys
if "../src" not in sys.path:
    sys.path.append("../src")

In [6]:
import torch
from transformers import MT5Config, MT5TokenizerFast
from vec4gloss.vec4gloss_model import Vec4GlossModel

In [3]:
tokenizer = MT5TokenizerFast.from_pretrained("google/mt5-small")

C:\Python310\lib\site-packages\transformers\convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
model = Vec4GlossModel(MT5Config.from_pretrained("google/mt5-small"))

In [36]:
batch = tokenizer(["這是一本書", "沒有人會來"], return_tensors="pt", padding=True)
batch["start_markers"] = torch.tensor([1,2])
batch["end_markers"] = torch.tensor([2,4])
batch["labels"] = tokenizer(["初一", "十五"], return_tensors="pt", padding=True)["input_ids"]
batch["decoder_input_ids"] = tokenizer(["<pad>初一", "<pad>十五"], 
                                       return_tensors="pt", padding=True)["input_ids"][:,:-1]
batch

{'input_ids': tensor([[   259, 222077,   1374, 200001,      1,      0],
        [   259,  98173,   1193,  17481,  21270,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]]), 'start_markers': tensor([1, 2]), 'end_markers': tensor([2, 4]), 'labels': tensor([[   259,  12869,   1374,      1],
        [   259, 146581,      1,      0]]), 'decoder_input_ids': tensor([[     0,    259,  12869,   1374],
        [     0,    259, 146581,      1]])}

In [56]:
out, enc_last_hiddens = model(**batch)

In [57]:
model.zero_grad()

In [58]:
out.loss.backward()

In [59]:
# looks like the gradient doesn't break
list(model.encoder.named_parameters())[3][1].grad

tensor([[ 0.5075,  0.0648, -0.4992,  ..., -0.2714, -0.4884,  0.0903],
        [-0.4077,  0.5823,  1.0992,  ..., -0.1347, -0.1547,  0.0479],
        [ 0.8445, -0.5915, -1.0442,  ..., -0.7448,  0.6044,  0.1048],
        ...,
        [ 0.6859, -0.9926, -1.7471,  ...,  0.2514, -0.3003,  0.0679],
        [ 0.4580, -0.3477, -0.7394,  ...,  0.5804, -0.0328, -0.1260],
        [ 0.2325, -0.5248, -0.2878,  ..., -1.0013, -0.3535,  0.2592]])